Nous allons dans ce notebook ajouter des données via l'API Yelp.

In [1]:
import requests
import json
import pandas as pd

In [2]:
# Récupère l'api key
with open('key_api_yelp.txt') as f:
    lines = f.readlines()

api_key = lines[0]

headers = {'Authorization': 'Bearer %s' % api_key}

In [3]:
# Définir les en-têtes et les urls de l'endpoint
headers = {'Authorization': 'Bearer %s' % api_key,
           'Content-Type':'application/json'}
url = 'https://api.yelp.com/v3/businesses'

In [4]:
%%time
# Créer une liste pour stocker le résultat
resto_ids = []

# Définir un dictionnaire de paramètres
endpoint_1 = url + '/search'
params = {'term':'restaurants',
          'location':'Paris, France',
          'offset':0,
          'limit':50}

current_offset=0
current_limit=50 
total_response = 200

while current_offset <= total_response:
    print(current_offset)
    params['offset'] = current_offset
    response = requests.get(url=endpoint_1,
                            headers=headers,
                            params=params)
    
    # S'assure que nous avons un identifiant unique pour l'entreprise
    # .json décode le format en objet Python (dictionnaire)
    if 'businesses' in response.json():
        # Sauvegarde les identifiants en les ajoutant aux précédents
        resto_ids += [business['id'] for business in response.json()['businesses']]
    current_offset += current_limit

0
50
100
150
200
CPU times: user 174 ms, sys: 4.67 ms, total: 178 ms
Wall time: 9.04 s


In [12]:
len(resto_ids)

250

In [ ]:
%%time
# Créer le dataframe qui recevra les données
df_review = pd.DataFrame(columns=['review_id',
                             'business_id',
                             'rating',
                             'text'])
# Récupérer les commentaires associés
i=0
for x in resto_ids:
    # Place l'url complète avec l'id du restaurant dans la varirable
    # endpoint_2
    endpoint_2 = url + '/' + str(x) + "/reviews"
    response = requests.get(url=endpoint_2,
                            headers=headers)
    
    # response.json ne possède que 3 lignes à la fois
    for review in response.json()['reviews']:
        df_review.loc[i]=[review['id'],
                     x,
                     review['rating'],
                     review['text']]
        i+=1

In [6]:
print(df_review.shape)
df_review.head()

(736, 4)


,review_id,business_id,rating,text
0,ujgk_YPp0MlHZQT6EEv-2A,-0iLH7iQNYtoURciDpJf6w,5,Some of the tastiest escargots I've had to dat...
1,nrxFK8QgnGU3zL4wTSB0PA,-0iLH7iQNYtoURciDpJf6w,2,I went back and forth trying to figure out wha...
2,r3bxIJ2ekrp8UPseAj2wjQ,-0iLH7iQNYtoURciDpJf6w,5,"What an amazing Bistro!! First off, even if yo..."
3,pjgcPURNS2PvAQ6a0LOShw,IU9_wVOGBKjfqTTpAXpKcQ,5,"You cannot beat this spot for food, drinks, an..."
4,G-YvPO-wUd-NIJDrahcYUg,IU9_wVOGBKjfqTTpAXpKcQ,4,what a wonderful lunch! \nthe decor is so fren...


In [7]:
# Export la dataframe dans un fichier CSV
df_review.to_csv('data/yelp_dataset/df_review_api.csv',
            index=False,
            encoding='utf-8')